In [283]:
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.layers import LSTM, Dense, SimpleRNN

In [284]:
A = Image.open('A.png').convert('1')
A2 = Image.open('A2.png').convert('1')
D = Image.open('D.png').convert('1')
D2 = Image.open('D2.png').convert('1')
V = Image.open('V.png').convert('1')
V2 = Image.open('V2.png').convert('1')

In [285]:
A_noisy = Image.open('A_noisy.png').convert('1')
D_noisy = Image.open('D_noisy.png').convert('1')
V_noisy = Image.open('V_noisy.png').convert('1')
A_test_noisy = Image.open('A_test.png').convert('1')
V_test_noisy = Image.open('V_test.png').convert('1')

In [286]:
C = Image.open('C.png').convert('1')

In [287]:
A_vec = np.array(A).reshape((1, -1))[0]
A2_vec = np.array(A2).reshape((1, -1))[0]
D_vec = np.array(D).reshape((1, -1))[0]
D2_vec = np.array(D2).reshape((1, -1))[0]
V_vec = np.array(V).reshape((1, -1))[0]
V2_vec = np.array(V2).reshape((1, -1))[0]

In [288]:
X_train = np.vstack((A_vec, D_vec, V_vec, A2_vec, D2_vec, V2_vec))
y_train = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1]])


In [289]:
#X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])

In [290]:
A_noisy_vec = np.array(A_noisy).reshape((1, -1))
D_noisy_vec = np.array(D_noisy).reshape((1, -1))
V_noisy_vec = np.array(V_noisy).reshape((1, -1))
A_test_vec = np.array(A_test_noisy).reshape((1, -1))
V_test_vec = np.array(V_test_noisy).reshape((1, -1))
C_vec = np.array(C).reshape((1, -1))

In [291]:
X_test = np.vstack((A_noisy_vec, D_noisy_vec, V_noisy_vec, A_test_vec, V_test_vec))
y_test = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1]])

In [292]:
# model = Sequential() model.add(Dense(units=121, input_dim=121, activation='sigmoid')) model.add(Dense(units=121, activation='relu')) model.add(Dense(units=121, activation='tanh')) model.add(Dense(units=121, activation='sigmoid')) model.add(Dense(units=121, activation='relu')) model.add(Dense(units=3, activation='softmax'))

In [293]:
A

In [294]:
model = Sequential()
model.add(SimpleRNN(units=121, input_shape=(1, 121), activation='sigmoid', return_sequences=True))
model.add(SimpleRNN(units=121, input_shape=(1, 121), activation='sigmoid', return_sequences=True))
model.add(Flatten())
model.add(Dense(units=3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=1)


Epoch 1/100


ValueError: in user code:

    File "c:\Users\ReyLegar\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\ReyLegar\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\ReyLegar\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\ReyLegar\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\ReyLegar\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File

In [ ]:
#X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [ ]:
y_pred = model.predict(X_test)

1/1 [==============================] - 0s 177ms/step


In [ ]:
y_pred_classes = np.argmax(y_pred, axis=1)

In [ ]:
print(y_pred)

[[0.91593    0.07257061 0.01149937]
 [0.0305779  0.9517983  0.01762377]
 [0.04446258 0.02702882 0.9285086 ]
 [0.7701253  0.16521357 0.06466106]
 [0.15323003 0.43397823 0.41279173]]


In [ ]:
for i in range(len(y_pred_classes)):
    if y_pred_classes[i] == 0:
        print('A')
    elif y_pred_classes[i] == 1:
        print('D')
    elif y_pred_classes[i] == 2:
        print('V')
    else:
        print('Unknown')

A
D
V
A
D


In [295]:
import tensorflow as tf
import numpy as np

class HammingNeuralNetwork(tf.keras.Model):
    def __init__(self, num_classes, num_hamming_layers, num_neurons):
        super(HammingNeuralNetwork, self).__init__()
        self.num_classes = num_classes
        self.num_hamming_layers = num_hamming_layers
        self.num_neurons = num_neurons
        
        self.dense_layers = [tf.keras.layers.Dense(num_neurons, activation='relu') for i in range(num_hamming_layers)]
        self.output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        x = tf.keras.layers.Flatten()(inputs)
        
        for dense_layer in self.dense_layers:
            x = dense_layer(x)
            x = self.hamming(x)
        
        x = self.output_layer(x)
        return x
    
    def hamming(self, inputs):
        x = tf.math.sign(inputs)
        x = tf.reduce_sum(x, axis=1, keepdims=True)
        x = tf.where(tf.math.equal(x, 0), tf.ones_like(x), x)
        
        x = tf.math.divide(inputs, x)
        return x

model = HammingNeuralNetwork(num_classes=5, num_hamming_layers=2, num_neurons=16)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=1, epochs=10, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

y_pred = model.predict(X_test)
print('Predicted label:', np.argmax(y_pred))


Epoch 1/10


ValueError: in user code:

    File "c:\Users\ReyLegar\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\ReyLegar\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\ReyLegar\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\ReyLegar\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\ReyLegar\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "